In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import json

from selenium import webdriver
from selenium_stealth import stealth
from selenium.webdriver.common.by import By

from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

import elo_config

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)


class ParsingData:
    def __init__(self):
        self.options = webdriver.ChromeOptions()
        self.options.add_argument("start-maximized")
        self.options.add_experimental_option("excludeSwitches", ["enable-automation"])
        self.options.add_experimental_option("useAutomationExtension", False)
        self.options.add_argument("--disable-notifications")
        self.driver = webdriver.Chrome(
            options=self.options,
            executable_path=r".\\scraping\\chromedriver.exe",
            service=Service(ChromeDriverManager().install()),
        )
        stealth(
            self.driver,
            languages=["en-US", "en"],
            vendor="Google Inc.",
            platform="Win32",
            webgl_vendor="Intel Inc.",
            renderer="Intel Iris OpenGL Engine",
            fix_hairline=True,
        )

        self.headers = {
            "Accept": "*/*",
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36",
        }


def replace_empty_to_none(data_list):
    new_list = []
    for item in data_list:
        new_item = {key: (None if value == '' else value) for key, value in item.items()}
        new_list.append(new_item)
    return new_list

    

def get_matches_by_elo():
    elo = ParsingData()
    web = elo_config.ELO_WEB_BY_COUNTRY.format(country="ENG", season="2022-2023")
    elo.driver.get(url=web)
    # page = elo.driver.find_element(By.TAG_NAME, 'body')

    req = requests.get(url=web, headers=elo.headers)
    src = req.text
    soup = BeautifulSoup(src, "lxml")
    country = soup.find(id="ranking").find("h3").find("b").text.strip()
    table_data = soup.find(id="Ranking").find(id="body").find("table").find_all("tr")
    elo_data_table = []
    count = 0
    for item in table_data:
        elo_data = item.find_all("td")
        if elo_data != []:
            table_id = elo_data[0].text.strip()
            Team = elo_data[1].text.strip()
            F1 = elo_data[2].text.strip()
            F2 = elo_data[3].text.strip()
            F3 = elo_data[4].text.strip()
            F4 = elo_data[5].text.strip()
            F5 = elo_data[6].text.strip()
            F6 = elo_data[7].text.strip()
            rating = elo_data[8].text.strip()
            record = elo_data[9].text.strip()
            all_time = elo_data[10].text.strip()
            diff = elo_data[11].text.strip()
            Y1 = elo_data[12].text.strip()
            season_to_date = elo_data[13].text.strip()
            league = elo_data[14].text.strip()
            cup = elo_data[15].text.strip()
            europe = elo_data[16].text.strip()
            uefa = elo_data[17].text.strip()
            di = {
                "table_id": table_id,
                "country": country,
                "Team": Team,
                "F1": F1,
                "F2": F2,
                "F3": F3,
                "F4": F4,
                "F5": F5,
                "F6": F6,
                "rating": rating,
                "record": record,
                "all_time": all_time,
                "diff": diff,
                "Y1": Y1,
                "season_to_date": season_to_date,
                "league": league,
                "cup": cup,
                "europe": europe,
                "uefa": uefa,
            }
            elo_data_table.append(di)
            count = count + 1
        else:
            continue

    elo_data_table_with_none = replace_empty_to_none(elo_data_table)
    df = pd.DataFrame(elo_data_table_with_none)
    
    elo.driver.close()

    return df




In [2]:
l = get_matches_by_elo()


C:\Users\darju\AppData\Local\Temp\ipykernel_10084\3789820017.py:29: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  self.driver = webdriver.Chrome(


In [8]:
import pandas as pd

url = "http://elofootball.com/country.php?countryiso=ENG&season=2022-2023"

# Pobierz stronę internetową i przekształć ją w DataFrame
tables = pd.read_html(url)

# Wybierz odpowiednią tabelę (może być wiele tabel)
# Jeśli wiesz, że jest tylko jedna tabela na stronie, to możesz ją wybrać tak: df = tables[0]
#df = tables[1]  # Zakładam, że interesuje Cię druga tabela na stronie

# Teraz masz tabelę w formacie DataFrame i możesz z nią pracować



In [ ]:
def parsing_elo_matches(country, season):
    url = elo_config.ELO_WEB_BY_COUNTRY.format(country=country, season=season)

    tables = pd.read_html(url)

    return table_data

In [13]:
tables[4]

,Date,Competition,Unnamed: 2,Home,Unnamed: 4,→,Unnamed: 6,Result,Probabilities: H,D,A,Away,Unnamed: 12,→.1,Unnamed: 14,Unnamed: 15
0,2023/06/10,Champions League,n,Manchester City,2392,9,2401,1 - 0,53,25,21,Inter,2231,-9,2222,NaN
1,2023/06/07,Europa Conf. League,n,ACF Fiorentina,2022,-10,2012,1 - 2,35,29,35,West Ham United,2021,10,2031,NaN
2,2023/06/03,FA Cup,n,Manchester City,2388,4,2392,2 - 1,60,23,17,Manchester United,2170,-4,2166,NaN
3,2023/05/29,League One,n,Barnsley FC,1645,-8,1637,0 - 1,29,29,42,Sheffield Wednesday,1706,8,1714,NaN
4,2023/05/28,Premier League,NaN,Brentford FC,2052,16,2068,1 - 0,15,21,64,Manchester City,2404,-16,2388,NaN
5,2023/05/28,Premier League,NaN,Chelsea FC,2037,0,2037,1 - 1,37,29,34,Newcastle United,2125,0,2125,NaN
6,2023/05/28,Premier League,NaN,Aston Villa,2056,8,2064,2 - 1,41,29,30,Brighton & Hove Albion,2101,-8,2093,NaN
7,2023/05/28,Premier League,NaN,Manchester United,2167,3,2170,2 - 1,68,19,13,Fulham FC,1973,-3,1970,NaN
8,2023/05/28,Premier League,NaN,Arsenal FC,2176,5,2181,5 - 0,71,18,11,Wolverhampton Wanderers,1957,-5,1952,NaN
9,2023/05/28,Premier League,NaN,Southampton FC,1828,7,1835,4 - 4,13,19,69,Liverpool FC,2223,-7,2216,NaN
